<a href="https://colab.research.google.com/github/mangeshb1/Gen_AI_And_NLP_Experiments/blob/main/Gen_AI_And_Chatbots/RAG_with_Gemma_Models_And_MongoDB_as_VDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!nvidia-smi

Sun Feb 25 14:53:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0              29W /  70W |   1501MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
### Notebook Based on https://www.youtube.com/watch?v=BNUpRW-Dk90
## and https://www.mongodb.com/developer/products/atlas/gemma-mongodb-huggingface-rag/
## Please note you will need working MongoDB - Vector Store for this code to work
__author__ = "Mangesh (MJ) B"
__copyright__ = "Copyright (C) 2024 Mangesh Bhangare"
__version__ = "1.0"

In [8]:
!pip install datasets pandas pymongo sentence_transformers
!pip install -U transformers
# Install below if using GPU
!pip install accelerate

In [9]:
# Load Dataset
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/AIatMongoDB/embedded_movies

## This dataset also has pre-calculated embeddings as well. Embeddings are based on Open AI's models.
dataset = load_dataset("AIatMongoDB/embedded_movies")

# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset["train"])

dataset_df.head(5)

,metacritic,num_mflix_comments,type,directors,countries,plot_embedding,fullplot,title,runtime,imdb,rated,genres,cast,poster,writers,plot,languages,awards
0,NaN,0,movie,"[Louis J. Gasnier, Donald MacKenzie]",[USA],"[0.00072939653, -0.026834568, 0.013515796, -0....",Young Pauline is left a lot of money when her ...,The Perils of Pauline,199.0,"{'id': 4465, 'rating': 7.6, 'votes': 744}",None,[Action],"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",https://m.media-amazon.com/images/M/MV5BMzgxOD...,"[Charles W. Goddard (screenplay), Basil Dickey...",Young Pauline is left a lot of money when her ...,[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
1,NaN,0,movie,"[Alfred J. Goulding, Hal Roach]",[USA],"[-0.022837115, -0.022941574, 0.014937485, -0.0...",As a penniless man worries about how he will m...,From Hand to Mouth,22.0,"{'id': 10146, 'rating': 7.0, 'votes': 639}",TV-G,"[Comedy, Short, Action]","[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",https://m.media-amazon.com/images/M/MV5BNzE1OW...,[H.M. Walker (titles)],A penniless young man tries to save an heiress...,[English],"{'nominations': 1, 'text': '1 nomination.', 'w..."
2,NaN,0,movie,[Herbert Brenon],[USA],"[0.00023330493, -0.028511643, 0.014653289, -0....","Michael ""Beau"" Geste leaves England in disgrac...",Beau Geste,101.0,"{'id': 16634, 'rating': 6.9, 'votes': 222}",None,"[Action, Adventure, Drama]","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",None,"[Herbert Brenon (adaptation), John Russell (ad...","Michael ""Beau"" Geste leaves England in disgrac...",[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
3,NaN,1,movie,[Albert Parker],[USA],"[-0.005927917, -0.033394486, 0.0015323418, -0....",A nobleman vows to avenge the death of his fat...,The Black Pirate,88.0,"{'id': 16654, 'rating': 7.2, 'votes': 1146}",None,"[Adventure, Action]","[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",https://m.media-amazon.com/images/M/MV5BMzU0ND...,"[Douglas Fairbanks (story), Jack Cunningham (a...","Seeking revenge, an athletic young man joins t...",None,"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
4,NaN,0,movie,[Sam Taylor],[USA],"[-0.0059373598, -0.026604708, -0.0070914757, -...","The Uptown Boy, J. Harold Manners (Lloyd) is a...",For Heaven's Sake,58.0,"{'id': 16895, 'rating': 7.6, 'votes': 918}",PASSED,"[Action, Comedy, Romance]","[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",https://m.media-amazon.com/images/M/MV5BMTcxMT...,"[Ted Wilde (story), John Grey (story), Clyde B...",An irresponsible young millionaire changes his...,[English],"{'nominations': 1, 'text': '1 nomination.', 'w..."


In [10]:
# Data Preparation

# Remove data point where plot coloumn is missing
dataset_df = dataset_df.dropna(subset=["fullplot"])
print("\nNumber of missing values in each column after removal:")
print(dataset_df.isnull().sum())

# Remove the plot_embedding from each data point in the dataset as we are going to create new embeddings with an open source embedding model from Hugging Face
dataset_df = dataset_df.drop(columns=["plot_embedding"])
dataset_df.head(5)


Number of missing values in each column after removal:
metacritic            893
num_mflix_comments      0
type                    0
directors              12
countries               0
plot_embedding          1
fullplot                0
title                   0
runtime                14
imdb                    0
rated                 279
genres                  0
cast                    1
poster                 78
writers                13
plot                    0
languages               1
awards                  0
dtype: int64


,metacritic,num_mflix_comments,type,directors,countries,fullplot,title,runtime,imdb,rated,genres,cast,poster,writers,plot,languages,awards
0,NaN,0,movie,"[Louis J. Gasnier, Donald MacKenzie]",[USA],Young Pauline is left a lot of money when her ...,The Perils of Pauline,199.0,"{'id': 4465, 'rating': 7.6, 'votes': 744}",None,[Action],"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",https://m.media-amazon.com/images/M/MV5BMzgxOD...,"[Charles W. Goddard (screenplay), Basil Dickey...",Young Pauline is left a lot of money when her ...,[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
1,NaN,0,movie,"[Alfred J. Goulding, Hal Roach]",[USA],As a penniless man worries about how he will m...,From Hand to Mouth,22.0,"{'id': 10146, 'rating': 7.0, 'votes': 639}",TV-G,"[Comedy, Short, Action]","[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",https://m.media-amazon.com/images/M/MV5BNzE1OW...,[H.M. Walker (titles)],A penniless young man tries to save an heiress...,[English],"{'nominations': 1, 'text': '1 nomination.', 'w..."
2,NaN,0,movie,[Herbert Brenon],[USA],"Michael ""Beau"" Geste leaves England in disgrac...",Beau Geste,101.0,"{'id': 16634, 'rating': 6.9, 'votes': 222}",None,"[Action, Adventure, Drama]","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",None,"[Herbert Brenon (adaptation), John Russell (ad...","Michael ""Beau"" Geste leaves England in disgrac...",[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
3,NaN,1,movie,[Albert Parker],[USA],A nobleman vows to avenge the death of his fat...,The Black Pirate,88.0,"{'id': 16654, 'rating': 7.2, 'votes': 1146}",None,"[Adventure, Action]","[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",https://m.media-amazon.com/images/M/MV5BMzU0ND...,"[Douglas Fairbanks (story), Jack Cunningham (a...","Seeking revenge, an athletic young man joins t...",None,"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
4,NaN,0,movie,[Sam Taylor],[USA],"The Uptown Boy, J. Harold Manners (Lloyd) is a...",For Heaven's Sake,58.0,"{'id': 16895, 'rating': 7.6, 'votes': 918}",PASSED,"[Action, Comedy, Romance]","[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",https://m.media-amazon.com/images/M/MV5BMTcxMT...,"[Ted Wilde (story), John Grey (story), Clyde B...",An irresponsible young millionaire changes his...,[English],"{'nominations': 1, 'text': '1 nomination.', 'w..."


In [11]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")


def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)
    # Numpy Array to Python List
    return embedding.tolist()


dataset_df["embedding"] = dataset_df["fullplot"].apply(get_embedding)

dataset_df.head()

,metacritic,num_mflix_comments,type,directors,countries,fullplot,title,runtime,imdb,rated,genres,cast,poster,writers,plot,languages,awards,embedding
0,NaN,0,movie,"[Louis J. Gasnier, Donald MacKenzie]",[USA],Young Pauline is left a lot of money when her ...,The Perils of Pauline,199.0,"{'id': 4465, 'rating': 7.6, 'votes': 744}",None,[Action],"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",https://m.media-amazon.com/images/M/MV5BMzgxOD...,"[Charles W. Goddard (screenplay), Basil Dickey...",Young Pauline is left a lot of money when her ...,[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[-0.009285839274525642, -0.005062091629952192,..."
1,NaN,0,movie,"[Alfred J. Goulding, Hal Roach]",[USA],As a penniless man worries about how he will m...,From Hand to Mouth,22.0,"{'id': 10146, 'rating': 7.0, 'votes': 639}",TV-G,"[Comedy, Short, Action]","[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",https://m.media-amazon.com/images/M/MV5BNzE1OW...,[H.M. Walker (titles)],A penniless young man tries to save an heiress...,[English],"{'nominations': 1, 'text': '1 nomination.', 'w...","[-0.002439370146021247, 0.023095937445759773, ..."
2,NaN,0,movie,[Herbert Brenon],[USA],"Michael ""Beau"" Geste leaves England in disgrac...",Beau Geste,101.0,"{'id': 16634, 'rating': 6.9, 'votes': 222}",None,"[Action, Adventure, Drama]","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",None,"[Herbert Brenon (adaptation), John Russell (ad...","Michael ""Beau"" Geste leaves England in disgrac...",[English],"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[0.012204294092953205, -0.011455751955509186, ..."
3,NaN,1,movie,[Albert Parker],[USA],A nobleman vows to avenge the death of his fat...,The Black Pirate,88.0,"{'id': 16654, 'rating': 7.2, 'votes': 1146}",None,"[Adventure, Action]","[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",https://m.media-amazon.com/images/M/MV5BMzU0ND...,"[Douglas Fairbanks (story), Jack Cunningham (a...","Seeking revenge, an athletic young man joins t...",None,"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[0.004541350528597832, -0.0006100559257902205,..."
4,NaN,0,movie,[Sam Taylor],[USA],"The Uptown Boy, J. Harold Manners (Lloyd) is a...",For Heaven's Sake,58.0,"{'id': 16895, 'rating': 7.6, 'votes': 918}",PASSED,"[Action, Comedy, Romance]","[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",https://m.media-amazon.com/images/M/MV5BMTcxMT...,"[Ted Wilde (story), John Grey (story), Clyde B...",An irresponsible young millionaire changes his...,[English],"{'nominations': 1, 'text': '1 nomination.', 'w...","[-0.002225600415840745, 0.011567802168428898, ..."


In [ ]:
## First setup MONGO_URI with login config in the notebook's secret
# https://cloud.mongodb.com/v2

In [14]:
import pymongo
from google.colab import userdata


def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None


mongo_uri = userdata.get("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client["movies"]
collection = db["movie_collection_2"]

Connection to MongoDB successful


In [15]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff00000000000000f4'), 'opTime': {'ts': Timestamp(1708873084, 30), 't': 244}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708873084, 30), 'signature': {'hash': b'\x1bb\\\x9d\x94\x84\xaf\xba\xb0\x9a.4\x83\xd0\x06C\x8f\x13\xa8\xda', 'keyId': 7285440633500598274}}, 'operationTime': Timestamp(1708873084, 30)}, acknowledged=True)

In [16]:
documents = dataset_df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


In [17]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 4,  # Return top 4 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "fullplot": 1,  # Include the plot field
                "title": 1,  # Include the title field
                "genres": 1,  # Include the genres field
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
        },
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

In [18]:
def get_search_result(query, collection):

    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('fullplot', 'N/A')}\n"

    return search_result

In [19]:
# Conduct query with retrival of sources
query = "What is the best Sci-Fi movie to watch and why?"
source_information = get_search_result(query, collection)
combined_information = f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}."

print(combined_information)

Query: What is the best Sci-Fi movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Naechureol siti, Plot: In the year 2080, the world is connected by a massive computer network. Combiners have developed a process that allows them to merge the souls of human and machine/cyborg, wreaking havoc in a deeply divided society. This movie shares traits with other films like Blade Runner and Ghost in the shell. A visually impressive piece, with a strong emphasis on relationships. Genre - Sci-Fi, Dystopian.
Title: Red Planet, Plot: In the near future, Earth is dying. A new colony on Mars could be humanity's only hope. A team of American astronauts, each a specialist in a different field, is making the first manned expedition to the red planet and must struggle to overcome the differences in their personalities, backgrounds and ideologies for the overall good of the mission. When their equipment suffers life-threatening damage and the crew must depend on one a

In [21]:
from huggingface_hub import notebook_login
notebook_login()

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# CPU Enabled uncomment below 👇🏽
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [23]:
# Moving tensors to GPU
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=500)
print(tokenizer.decode(response[0]))

<bos>Query: What is the best Sci-Fi movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Naechureol siti, Plot: In the year 2080, the world is connected by a massive computer network. Combiners have developed a process that allows them to merge the souls of human and machine/cyborg, wreaking havoc in a deeply divided society. This movie shares traits with other films like Blade Runner and Ghost in the shell. A visually impressive piece, with a strong emphasis on relationships. Genre - Sci-Fi, Dystopian.
Title: Red Planet, Plot: In the near future, Earth is dying. A new colony on Mars could be humanity's only hope. A team of American astronauts, each a specialist in a different field, is making the first manned expedition to the red planet and must struggle to overcome the differences in their personalities, backgrounds and ideologies for the overall good of the mission. When their equipment suffers life-threatening damage and the crew must depend on 